In [ ]:
import sys
import datetime

import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt

sys.path.insert(0,'..')
import covid19_inference as cov19

confirmed_cases = cov19.get_jhu_confirmed_cases()

country = 'Germany'
date_data_begin = datetime.datetime(2020,3,1)
date_data_end   = cov19.get_last_date(confirmed_cases)

#date_data_end   = datetime.datetime(2020,3,28)
num_days_data = (date_data_end-date_data_begin).days
diff_data_sim = 16 # should be significantly larger than the expected delay, in 
                   # order to always fit the same number of data points.
num_days_future = 28
date_begin_sim = date_data_begin - datetime.timedelta(days = diff_data_sim)
date_end_sim   = date_data_end   + datetime.timedelta(days = num_days_future)
num_days_sim = (date_end_sim-date_begin_sim).days


cases_obs = cov19.filter_one_country(confirmed_cases, country,
                                     date_data_begin, date_data_end)

change_points = [dict(prior_mean_date_begin_transient = datetime.datetime(2020,3,9),
                      prior_median_λ = 0.2),
                 dict(prior_mean_date_begin_transient = datetime.datetime(2020,3,16),
                      prior_sigma_date_begin_transient = 1,
                      prior_median_λ = 1/8,
                      prior_sigma_λ = 0.2),
                 dict(prior_mean_date_begin_transient = datetime.datetime(2020,3,23),
                      prior_sigma_date_begin_transient = 1,
                      prior_median_λ = 1/8/2,
                      prior_sigma_λ = 0.2)]

model = cov19.SIR_model_with_change_points(np.diff(cases_obs),
                                                       change_points,
                                                       date_begin_sim,
                                                       num_days_sim,
                                                       diff_data_sim)

trace = pm.sample(model=model, init='advi')
